# Dependencies

In [ ]:
!pip install tensorflow-text

In [66]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import re
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn import metrics

# Data Preparation

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
file_path = '/content/gdrive/MyDrive/FAIZ/Dataset/Workout recom/megaGymDataset.csv'

In [4]:
data = pd.read_csv(file_path)
data.head()

,Unnamed: 0,Title,Desc,Type,BodyPart,Equipment,Level,Rating,RatingDesc
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0,NaN
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,NaN,NaN
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,NaN,NaN


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2918 entries, 0 to 2917
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  2918 non-null   int64  
 1   Title       2918 non-null   object 
 2   Desc        1368 non-null   object 
 3   Type        2918 non-null   object 
 4   BodyPart    2918 non-null   object 
 5   Equipment   2918 non-null   object 
 6   Level       2918 non-null   object 
 7   Rating      1031 non-null   float64
 8   RatingDesc  862 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 205.3+ KB


In [9]:
data = data.drop(['Unnamed: 0', 'Rating', 'RatingDesc'], axis=1)

In [10]:
data = data.dropna(subset=['Desc'])

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1368 entries, 0 to 2916
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Title      1368 non-null   object
 1   Desc       1368 non-null   object
 2   Type       1368 non-null   object
 3   BodyPart   1368 non-null   object
 4   Equipment  1368 non-null   object
 5   Level      1368 non-null   object
dtypes: object(6)
memory usage: 74.8+ KB


In [20]:
data.value_counts('Type')

Type
Strength                 1234
Plyometrics                53
Stretching                 44
Cardio                     17
Powerlifting               11
Olympic Weightlifting       5
Strongman                   4
dtype: int64

In [21]:
data.value_counts('BodyPart')

BodyPart
Abdominals     298
Quadriceps     245
Shoulders      174
Chest          149
Biceps         101
Triceps         88
Lats            69
Hamstrings      50
Middle Back     49
Lower Back      42
Glutes          29
Calves          26
Traps           17
Forearms        16
Abductors        8
Adductors        7
dtype: int64

In [22]:
data.value_counts('Equipment')

Equipment
Body Only        404
Dumbbell         246
Barbell          161
Cable            149
Machine          121
Other            101
Kettlebells       53
Bands             49
Exercise Ball     28
Medicine Ball     25
E-Z Curl Bar      14
None               9
Foam Roll          8
dtype: int64

In [23]:
data.value_counts('Level')

Level
Intermediate    1250
Beginner         108
Expert            10
dtype: int64

In [24]:
data.loc[data['Equipment'] == 'None']

,Title,Desc,Type,BodyPart,Equipment,Level
637,Decline oblique crunch,The decline oblique crunch is a popular bodywe...,Strength,Abdominals,None,Intermediate
638,Decline sit-up,The decline sit-up is a bodyweight core exerci...,Strength,Abdominals,None,Intermediate
639,Hanging Windshield Wiper,The hanging windshield wiper is an advanced ab...,Strength,Abdominals,None,Intermediate
1402,Glute ham raise-,The glute ham raise is an exercise targeting t...,Strength,Hamstrings,None,Beginner
1403,Lying hamstring stretch with band,The lying hamstring stretch with band is a sim...,Stretching,Hamstrings,None,Beginner
1406,Alternating lunge jump,The alternating lunge jump is an explosive bod...,Stretching,Hamstrings,None,Beginner
2421,Dumbbell lateral hop to sprint,The dumbbell lateral hop to sprint is a multi-...,Plyometrics,Quadriceps,None,Intermediate
2422,Smith machine lunge sprint,The Smith machine lunge sprint is a lower-body...,Strength,Quadriceps,None,Intermediate
2423,Sissy squat,The sissy squat is a bodyweight squat variatio...,Strength,Quadriceps,None,Intermediate


In [26]:
data = data.replace('None', "Body Only")

In [27]:
data.value_counts('Equipment')

Equipment
Body Only        413
Dumbbell         246
Barbell          161
Cable            149
Machine          121
Other            101
Kettlebells       53
Bands             49
Exercise Ball     28
Medicine Ball     25
E-Z Curl Bar      14
Foam Roll          8
dtype: int64

In [28]:
data.to_csv(r'/content/data.csv', index=False, header=True)

In [30]:
data.isna().sum()

Title        0
Desc         0
Type         0
BodyPart     0
Equipment    0
Level        0
dtype: int64

In [32]:
data.duplicated().sum()

9

In [44]:
data.loc[data.duplicated(subset=['Title'])]

,Title,Desc,Type,BodyPart,Equipment,Level
97,Decline bar press sit-up,The decline bar press sit-up is a weighted cor...,Strength,Abdominals,Barbell,Intermediate
645,Exercise Ball Cable Crunch - Gethin Variation,The exercise ball crunch is a popular gym exer...,Strength,Abdominals,Cable,Intermediate
958,Band-suspended kettlebell bench press,The band-suspended kettlebell bench press is a...,Strength,Chest,Bands,Intermediate
1730,Seated Cable Rows,The cable seated row is a popular exercise to ...,Strength,Middle Back,Cable,Intermediate
2004,Dumbbell step-up,The dumbbell step-up is a great exercise for b...,Strength,Quadriceps,Dumbbell,Intermediate
2655,Arnold press,Named after the iconic bodybuilder and movie s...,Strength,Shoulders,Dumbbell,Intermediate
2658,Seated rear delt fly,The seated rear delt fly is an upper-body exer...,Strength,Shoulders,Dumbbell,Intermediate


In [41]:
data = data.drop(index=939, axis=1)

In [43]:
data = data.drop(index=1709, axis=1)

# Modeling

In [46]:
new_data = pd.DataFrame()

In [48]:
new_data['workout'] = data['Title'].apply(lambda x: x.lower())
new_data['workout'] = new_data['workout'].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))

In [50]:
new_data['desc'] = data['Desc'].apply(lambda x: x.lower())
new_data['desc'] = new_data['desc'].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))

In [52]:
new_data['type'] = data['Type'].apply(lambda x: x.lower())
new_data['type'] = new_data['type'].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))

In [53]:
new_data['body_part'] = data['BodyPart'].apply(lambda x: x.lower())
new_data['body_part'] = new_data['body_part'].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))

In [54]:
new_data['equipment'] = data['Equipment'].apply(lambda x: x.lower())
new_data['equipment'] = new_data['equipment'].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))

In [55]:
new_data['difficulty'] = data['Level'].apply(lambda x: x.lower())
new_data['difficulty'] = new_data['difficulty'].apply(lambda x: re.sub('[^A-Za-z0-9]+', ' ', x))

In [56]:
new_data.head()

,workout,desc,type,body_part,equipment,difficulty
0,partner plank band row,the partner plank band row is an abdominal exe...,strength,abdominals,bands,intermediate
1,banded crunch isometric hold,the banded crunch isometric hold is an exercis...,strength,abdominals,bands,intermediate
2,fyr banded plank jack,the banded plank jack is a variation on the pl...,strength,abdominals,bands,intermediate
3,banded crunch,the banded crunch is an exercise targeting the...,strength,abdominals,bands,intermediate
4,crunch,the crunch is a popular core exercise targetin...,strength,abdominals,bands,intermediate


In [85]:
new_data.to_csv(r'/content/new_data.csv', index=False, header=True)

In [71]:
concat_data = pd.DataFrame()

In [78]:
concat_data['workout'] = new_data['workout']
concat_data['desc'] = 'type: ' + new_data['type'] + '; ' + 'target: ' + new_data['body_part'] + '; ' + 'using: ' + new_data['equipment'] + '; ' + 'difficulty: ' + new_data['difficulty'] + '; ' + new_data['desc']

In [79]:
concat_data.head()

,workout,desc
0,partner plank band row,type: strength; target: abdominals; using: ban...
1,banded crunch isometric hold,type: strength; target: abdominals; using: ban...
2,fyr banded plank jack,type: strength; target: abdominals; using: ban...
3,banded crunch,type: strength; target: abdominals; using: ban...
4,crunch,type: strength; target: abdominals; using: ban...


In [80]:
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=True)

def get_bert_embeddings(text, preprocessor, encoder):
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
  encoder_inputs = preprocessor(text_input)
  outputs = encoder(encoder_inputs)
  embedding_model = tf.keras.Model(text_input, outputs['pooled_output'])
  sentences = tf.constant([text])
  return embedding_model(sentences)

concat_data['encodings'] = concat_data['desc'].apply(lambda x: get_bert_embeddings(x, preprocessor, encoder))

In [83]:
def preprocess_text():
  text = input()
  text = text.lower()
  text = re.sub('[^A-Za-z0-9]+', ' ', text)
  return text
  
query_text = preprocess_text()
query_encoding = get_bert_embeddings(query_text, preprocessor, encoder)

concat_data['similarity_score'] = concat_data['encodings'].apply(lambda x: metrics.pairwise.cosine_similarity(x, query_encoding)[0][0])
results = concat_data.sort_values(by=['similarity_score'], ascending=False)

quads


In [84]:
results

,workout,desc,encodings,similarity_score
2741,bent over shoulder y,type: stretching; target: shoulders; using: bo...,"((tf.Tensor(-0.8767373, shape=(), dtype=float3...",0.984738
1494,reverse grip lat pull down,type: strength; target: lats; using: machine; ...,"((tf.Tensor(-0.8243555, shape=(), dtype=float3...",0.972299
1126,dumbbell push up row,type: strength; target: chest; using: dumbbell...,"((tf.Tensor(-0.77505887, shape=(), dtype=float...",0.969744
2067,leg press,type: strength; target: quadriceps; using: mac...,"((tf.Tensor(-0.79009414, shape=(), dtype=float...",0.968973
2103,up leg press,type: strength; target: quadriceps; using: mac...,"((tf.Tensor(-0.79009414, shape=(), dtype=float...",0.968973
...,...,...,...,...
2842,cable overhead triceps extension,type: strength; target: triceps; using: cable;...,"((tf.Tensor(0.099457525, shape=(), dtype=float...",-0.477494
1987,3d dumbbell lunge,type: strength; target: quadriceps; using: dum...,"((tf.Tensor(0.21795338, shape=(), dtype=float3...",-0.489511
2851,cable overhead triceps extension gethin variation,type: strength; target: triceps; using: cable;...,"((tf.Tensor(0.22808821, shape=(), dtype=float3...",-0.534999
2855,triceps overhead extension with rope gethin va...,type: strength; target: triceps; using: cable;...,"((tf.Tensor(0.22808821, shape=(), dtype=float3...",-0.534999
